In [89]:

import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pyautogui as pag
from time import sleep
import cv2

In [90]:
# Yakalanan yüz etrafına çizilecek çerçevenin renkleri tanımlandı.
RED = (0, 0, 255)
GREEN = (0, 255, 0)
BLUE = (255, 0, 0)
PINK = (255,0, 255)

In [91]:
# cv2 kütüphanesinden yüz tanıma üzerine eğitilmiş model alındı.
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


In [92]:
# Gözleri tahminleyen bizim eğittimiz model tanımlandı.
model_name = 'deneme_10_model.h5'
face_classifier = keras.models.load_model(f'{model_name}')

#Tahminde kullanılacak sınıflar tanımlandı.
class_names = ['down', 'enter','halt','up']

# Tahminler arası bekleme süresi tanımlandı. Göz açık konuma getirilene kadar hatalı işlem yapılması önlemek amaçlanmıştır.
delay = 0.35		

In [93]:
# Kameradan alınan görüntü tahmin yapmaya uygun hale gelmesi için işlenir.
def get_extended_image(img, x, y, w, h, k=0.1):
    if x - k*w > 0:
        start_x = int(x - k*w)
    else:
        start_x = x
    if y - k*h > 0:
        start_y = int(y - k*h)
    else:
        start_y = y

    end_x = int(x + (1 + k)*w)
    end_y = int(y + (1 + k)*h)

    face_image = img[start_y:end_y,
                     start_x:end_x]
    face_image = tf.image.resize(face_image, [224, 224])
    
    face_image = np.expand_dims(face_image, axis=0)
    return face_image

In [94]:
#Yakalanan yüz etrafına çerçeve çizilir. Yüzün koordinatları verilerek bu koorinatların üzerine çerçeve çizilir. Çerçeve üzerine tahmin ve doğruluk oranı yazdırılır.
def drawRectangle(x, y, w, h, color, prediction, frame , confidence):
    
        cv2.rectangle(frame,
                      (x, y), 
                      (x+w, y+h),
                      color,
                      2) 
        cv2.putText(frame,
        
                    "{:6} - {:.2f}%".format(prediction, confidence*100),
                    (x, y),
                    cv2.FONT_HERSHEY_PLAIN,  
                    2, 
                    color,
                    2) 

In [95]:
# Bu fonksiyon yapılan tahmine göre klavye tuşuna basılır. Orneğin tahmin edilen yüz "down" ise yani sağ göz kapalı isepag.press("down") ile alt ok tuşuna başılır.
def keyState(prediction):
    if prediction != 'halt':
        pag.press(prediction)
        sleep(delay)

In [96]:
video_capture = cv2.VideoCapture(0)  # webcamera
#video_capture = cv2.VideoCapture('http://192.168.43.1:4747/mjpegfeed')  # droidcam


if not video_capture.isOpened():
    print("Kameraya erisilemiyor.")
else:
    print("Kameraya başarili bir sekilde erisildi")

print("Goruntu akisi basladi - cikis icin ESC tusuna basiniz.")
while True:
    
    
    # Kameradan anlık olarak frameler alınır.
    ret, frame = video_capture.read()
    if not ret:
        print("Erisim yok ...")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Yüzlerin tespit edilmesi
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=5,
        minSize=(100, 100),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    # Tespit edilen her yüz için tahminleme ve yüz üzerine çerçeve çizilme işlemi yapılır.
    for (x, y, w, h) in faces:
        
        # yakalanan frame görüntü olarak alınır.
        face_image = get_extended_image(frame, x, y, w, h, 0.5)
        
        # Eğittiğimiz modele bu görüntü gönderilir. Sonuçların tahmin edilmesi yapılır.
        result = face_classifier.predict(face_image)
        
        # Tahmin edilen yüzün hangi sınıfa ait olduğu belirlenir.
        prediction = class_names[np.array( result[0]).argmax(axis=0)] 
        # Doğruluk oranı hesaplanır.
        confidence = np.array(result[0]).max(axis=0)
        # Tahmin sonucu belirlenen sınıfa göre belirli bir renk ve çerçeve üzerine yazdırılacak yazı belirlenir. 
        if prediction == 'down':
            color = GREEN
            cv2.putText(frame,"DOWN",(x,y),cv2.FONT_HERSHEY_COMPLEX,0.5,(250,250,250),1)
        elif prediction == 'up':
            color = BLUE
            cv2.putText(frame,"UP",(x,y),cv2.FONT_HERSHEY_COMPLEX,0.5,(250,250,250),1)
        elif prediction == 'enter':
            color = RED
            cv2.putText(frame,"ENTER",(x,y),cv2.FONT_HERSHEY_COMPLEX,0.5,(250,250,250),1)
        elif prediction == 'halt': 
            color = PINK
            cv2.putText(frame,"HALT",(x,y),cv2.FONT_HERSHEY_COMPLEX,0.5,(250,250,250),1)
            
        #Belirlenen renk ve yazı ile Yüz çevresine çerçeve çizilir.    
        drawRectangle(x, y, w, h, color, prediction, frame, confidence)
        
        
    # Gösterilen görüntü ekrana basılır.
    cv2.imshow("Face detector - cikis icin ESC tusuna basiniz.", frame)

    # ESC tusuna basıldığında program sonlandırılır.
    key = cv2.waitKey(1)
    if key % 256 == 27:  # ESC code
        break
    # Program çalıştığı sürece eğer gözler açık değilse yapılan hareket ile ilgili tuşa basılır. Her basma işleminde bir süre beklenir.
    #if prediction != 'halt':
    #   keyState(prediction)

video_capture.release()
cv2.destroyAllWindows()
print("Goruntu akisi sonlandi.")

Kameraya başarili bir sekilde erisildi
Goruntu akisi basladi - cikis icin ESC tusuna basiniz.
Goruntu akisi sonlandi.
